In [16]:
from seleniumbase import Driver
from bs4 import BeautifulSoup
import os
import json
import re
import tqdm

In [17]:
driver = Driver(
    browser="edge",
    uc=True,
    headless2=True,
    incognito=True,
    agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36 AVG/112.0.21002.139",
    do_not_track=True,
    undetectable=True,
)

In [18]:
PATH = "https://arxiv.org/" #Don't change

In [19]:
def run(URL, p = "results"):
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    tag = soup.find("div", class_ = "morefewer")
    tag = tag.find_all("a")[-1]['href'][1:]
    driver.get(PATH + tag)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    t = dict()
    r = []
    for ele in soup.find_all(["dt", "dd"]):
        if ele.name == "dt":
            a = ele.find_all("a")
            pdf = os.path.join(PATH, a[1]["href"][1:])
            t['pdf'] = pdf
        elif ele.name == "dd":
            authors = ele.find("div", class_ = "list-authors").find_all("a")
            title = ele.find("div", class_ = "list-title mathjax").text.strip()
            match = re.search(r'Title:\s+(.*)', title)
            if match:
                    title = match.group(1)
            t["title"] = title
            t["authors"] = [x.text for x in authors]
            r.append(t)
            t = dict()
    os.makedirs(p, exist_ok = True)
    nm = "-".join(URL.split("/")[-2:])
    with open(f"{p}/result_{nm}.json", "w") as f:
        json.dump(r, f, indent=4)

In [20]:
url = "https://arxiv.org/year/math-ph/2022" # Can change

parts = url.split('/')
if parts[-1].isdigit() and len(parts[-1]) == 4:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    content_div = soup.find('div', id='content')
    li = [os.path.join(PATH, x.find("a")["href"][1:]) for x in content_div.find("ul").find_all("li")]
    for l in tqdm.tqdm(li, "Processing"):
        run(l)
else:
    run(url)

Processing: 100%|██████████| 12/12 [00:15<00:00,  1.26s/it]
